# Feature Extraction (part 1)

In [ ]:
import librosa
import matplotlib.pyplot as plt
import vocalpy as voc

## Acoustic features

In their simplest form, these area scalar value we compute *per frame* (time bin) of a spectrogram. For example:
* amplitude, intensity, energy: the sum of values across the frequencies, e.g. the [Root Mean Square](https://librosa.org/doc/0.10.2/generated/librosa.feature.rms.html#librosa.feature.rms) of the signal
* entropy: the Shannon entropy of the  $H(x, sf) =  -\sum_{f=0}^{f_s/2} P(f) \log_2[P(f)]$
* fundamental frequency / pitch: typically computed with an algorithm such as [YIN](https://librosa.org/doc/0.10.2/generated/librosa.yin.html#librosa.yin)

In [ ]:
sound = voc.example('samba.wav', return_type="sound")

As an example we'll look at some features computed by [Sound Analysis Tools](http://soundanalysispro.com/matlab-sat).

The implementation in VocalPy is adapted from work by [Therese Koch](https://therese-koch.netlify.app/) in [AVN](https://github.com/theresekoch/avn/blob/main/avn/acoustics.py) and in [birdsonganalysis](https://github.com/PaulEcoffet/birdsonganalysis) by Paul Ecoffet.

We call the function `voc.feature.sat.similarity_features` with a `Sound`, and we get back a set of `vocalpy.Features`.

In [ ]:
features = voc.feature.sat.similarity_features(sound)

In [ ]:
features

In [ ]:
ftr_dat.data.data_vars

In [ ]:
spect = voc.spectrogram(sound)

fig, ax_arr = plt.subplots(3, 1, figsize=(6, 4.5), dpi=150)

voc.plot.spectrogram(spect, ax=ax_arr[0])
ftr_dat.data['amplitude'].plot(ax=ax_arr[1], color='b')
ftr_dat.data['frequency_modulation'].plot(ax=ax_arr[2], color='orange')
fig.tight_layout()

Another use of features is to reduce entire *segments* -- e.g., calls, syllables, etc.--to a 1-dimensional vector of *features*. This gives us a *feature space* we can use with many methods that expect vectors: machine learning classification, dimensionality reduction, distance measures.


## Precomputed Acoustic Features


As defined in [Elie Theunissen 2016](https://link.springer.com/article/10.1007/s10071-015-0933-6): a set of extracted features, in contrast with higher dimensionality "features". 

A similar approach is taken by the [warbleR](https://marce10.github.io/warbleR/index.html) function [`spectro_analysis`](https://marce10.github.io/warbleR/reference/spectro_analysis.html).

In [ ]:
wav_paths = voc.paths.from_dir(
    './data/Elie-Theunissen-2016-zebra-finch-song-library-subset/',
    'wav'
)

In [ ]:
data, samplerate = librosa.load(wav_paths[0])
data = librosa.to_mono(data)
sound = voc.Sound(data, samplerate)

Here we get the set of features computed by `soundsig`(https://github.com/theunissenlab/soundsig).

In [ ]:
# we skip computing fundamental frequency features since those are slower
out = voc.feature.soundsig.predefined_acoustic_features(sound, ftr_groups=("temporal", "spectral"))

Following the same pattern as before, we will use a `FeatureExtractor` class to extract features from many files.

In [ ]:
callback = voc.feature.soundsig.predefined_acoustic_features
params = dict(ftr_groups=("temporal", "spectral"))
extractor = voc.FeatureExtractor(callback, params)

In [ ]:
sounds = [voc.Sound.read(wav_path) for wav_path in wav_paths[:10]]
features_list = extractor.extract(sounds, parallelize=True)

In [ ]:
features_list[0]